# IBM Applied Data Science Capstone Project

## Week 5 Final Assignment Notebook

### Opening a New Restaurant in Hyderabad, India.

- Build a dataframe of neighborhoods in Hyderabad, India by web scraping the data from Wikipedia page


- Get the geographical coordinates of the neighborhoods


- Obtain the venue data for the neighborhoods from Foursquare API


- Explore and cluster the neighborhoods


- Select the best cluster to open a new Restaurant

### Import libraries:

In [1]:
!pip install geocoder
!pip install folium

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [4]:
#data

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
hy_df = pd.DataFrame({"Neighborhood": neighborhoodList})

hy_df.head()

,Neighborhood
0,A. C. Guards
1,A. S. Rao Nagar
2,Abhyudaya Nagar
3,Abids
4,Adibatla


This is the data frame created after scraping the data. We need to get the geographical coordinates in the form of latitude and longitude in order to be able to use Foursquare API. To do so, we will use the Geocoder package that will allow us to convert the address into geographical coordinates in the form of latitude and longitude.

In [9]:
# Defining a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in hy_df["Neighborhood"].tolist()]

We have obtained the latitude and longitude coordinates for all the places, so we need to merge the coordinates into the original data frame.

In [11]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# Merge the coordinates into the original dataframe
hy_df['Latitude'] = df_coords['Latitude']
hy_df['Longitude'] = df_coords['Longitude']

In [15]:
hy_df.head()

,Neighborhood,Latitude,Longitude
0,A. C. Guards,17.395015,78.459812
1,A. S. Rao Nagar,17.411200,78.508240
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adibatla,17.235790,78.541300


In [16]:
hy_df.shape

(200, 3)

This is the combined data frame which contains all the neighbourhoods along with the geographical coordinates.

In [18]:
# Getting the coordinates of Hyderabad
address = 'Hyderabad, India'
geolocator = Nominatim(user_agent="Hyderabad's mapping app", timeout=2)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.360589, 78.4740613.


After gathering the data, we have to populate the data into a pandas DataFrame and then visualize the neighbourhoods in a map using Folium package.

In [76]:
map_hy = folium.Map(location=[latitude, longitude], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(hy_df['Latitude'],  hy_df['Longitude'], hy_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_hy)
map_hy

In [22]:
# save the map as HTML file
map_hy.save(r"C:\Users\Hp\Desktop\P-jupyter\Others")

### Use the foursquare API to explore the neighbourhoods

In [23]:
CLIENT_ID = '2X1KKPYCKZ3COAHGCHROCBG2OWG5GO1JLS4L5DCVUSCMRTXI' # your Foursquare ID
CLIENT_SECRET = 'M5GUUMI0OV3LG31YVHMPENRYW4JTQ5X1PGMJI2DLKGBKWHCX' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2X1KKPYCKZ3COAHGCHROCBG2OWG5GO1JLS4L5DCVUSCMRTXI
CLIENT_SECRET:M5GUUMI0OV3LG31YVHMPENRYW4JTQ5X1PGMJI2DLKGBKWHCX


In [25]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(hy_df['Latitude'], hy_df['Longitude'], hy_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

After extracting all the venues, we have to convert the venues list into a new DataFrame.

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(6258, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. C. Guards,17.395015,78.459812,Cafe Niloufer & Bakers,17.399715,78.462881,Café
1,A. C. Guards,17.395015,78.459812,Subhan Bakery,17.392412,78.464712,Bakery
2,A. C. Guards,17.395015,78.459812,Chicha's,17.403255,78.460152,Hyderabadi Restaurant
3,A. C. Guards,17.395015,78.459812,Nizam club,17.403221,78.468729,Lounge
4,A. C. Guards,17.395015,78.459812,Taiba Bakers & Confectioners,17.402530,78.456823,Bakery
5,A. C. Guards,17.395015,78.459812,Rayalaseema Ruchulu,17.403084,78.463012,South Indian Restaurant
6,A. C. Guards,17.395015,78.459812,Shahi Dastarkhwan,17.402640,78.461399,Hyderabadi Restaurant
7,A. C. Guards,17.395015,78.459812,Laxman Ki Bandi,17.378895,78.463973,South Indian Restaurant
8,A. C. Guards,17.395015,78.459812,Siraj's International Juice Center,17.395579,78.442701,Juice Bar
9,A. C. Guards,17.395015,78.459812,Spice 6,17.409007,78.450559,Bistro


### Lets check how many venues were returned for each neighbourhood

In [29]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. C. Guards,70,70,70,70,70,70
A. S. Rao Nagar,20,20,20,20,20,20
Abhyudaya Nagar,9,9,9,9,9,9
Abids,79,79,79,79,79,79
Adikmet,19,19,19,19,19,19
Afzal Gunj,46,46,46,46,46,46
Aghapura,65,65,65,65,65,65
"Aliabad, Hyderabad",8,8,8,8,8,8
Alijah Kotla,16,16,16,16,16,16


### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 175 uniques categories.


In [32]:
venues_df['VenueCategory'].unique()[:50] #displays all the 175 category names

array(['Café', 'Bakery', 'Hyderabadi Restaurant', 'Lounge',
       'South Indian Restaurant', 'Juice Bar', 'Bistro', 'Science Museum',
       'Ice Cream Shop', 'Indian Restaurant', 'Park',
       'Middle Eastern Restaurant', 'Stadium', 'Performing Arts Venue',
       'Dessert Shop', 'Vegetarian / Vegan Restaurant', 'Hotel Bar',
       'Hotel', 'Shoe Store', 'Neighborhood', 'Diner', 'Snack Place',
       'Coffee Shop', 'Pizza Place', 'Mobile Phone Shop',
       'Fast Food Restaurant', 'Chinese Restaurant', 'Clothing Store',
       'Fried Chicken Joint', 'Department Store', 'Shopping Mall',
       'Soccer Field', 'Electronics Store', 'Hookah Bar',
       'Breakfast Spot', 'Planetarium', 'Movie Theater',
       'Convenience Store', 'Sandwich Place', 'Asian Restaurant',
       'Fruit & Vegetable Store', 'Restaurant', 'Food Truck',
       'Chaat Place', 'Burger Joint', 'Bar', 'Smoke Shop', 'Food',
       'Multiplex', 'Gaming Cafe'], dtype=object)

In [34]:
# check if the results contain "Restaurant"
"Restaurant" in venues_df['VenueCategory'].unique()

True

### Analyze each neighbourhood

Here we apply one hot encoding to all the venues. So now the number of columns becomes 175

In [35]:
# one hot encoding
hy_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hy_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hy_onehot.columns[-1]] + list(hy_onehot.columns[:-1])
hy_onehot = hy_onehot[fixed_columns]

print(hy_onehot.shape)
hy_onehot.head(20)

(6258, 176)


,Neighborhoods,ATM,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0

Next, let’s group rows of neighbourhood by taking the sum of the frequency of occurrence of each category.

In [39]:
hy_grouped=hy_onehot.groupby(["Neighborhoods"]).sum().reset_index()

hy_grouped

,Neighborhoods,ATM,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,2,1,1,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,2,0,0,2,3,0,12,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,2,1,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Abhyudaya Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abids,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,3,0,0,1,0,4,1,0,2,0,0,0,0,0,0,0,0,1,3,1,0,0,0,0,0,0,1,4,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,1,0,0,0,5,0,10,0,1,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,1,0,1,0,1,1,2,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Adikmet,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [41]:
print(hy_grouped.shape)

(197, 176)


In [42]:
rest = hy_grouped.loc[:, hy_grouped.columns.str.endswith('Restaurant')]
rest

,Afghan Restaurant,American Restaurant,Andhra Restaurant,Asian Restaurant,Bengali Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fondue Restaurant,French Restaurant,Greek Restaurant,Hunan Restaurant,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Multicuisine Indian Restaurant,New American Restaurant,North Indian Restaurant,Parsi Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,1,0,0,0,0,4,0,0,0,0,2,0,12,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,4,0,0,0,0,4,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,3,0,2,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0
6,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,2,0,10,0,0,0,1,0,0,0,0,0,0,1,0,3,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [47]:
rest.shape

(197, 33)

In [56]:
rest['Total_Restaurants']=rest.iloc[:,-33:].sum(axis=1)

rest

<ipython-input-56-8b95490b16a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest['Total_Restaurants']=rest.iloc[:,-33:].sum(axis=1)


,Afghan Restaurant,American Restaurant,Andhra Restaurant,Asian Restaurant,Bengali Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fondue Restaurant,French Restaurant,Greek Restaurant,Hunan Restaurant,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Multicuisine Indian Restaurant,New American Restaurant,North Indian Restaurant,Parsi Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Total_Restaurant,Fruit Total,rest Total,restTotal,Total_Restaurants
0,0,0,0,0,0,1,0,0,0,0,4,0,0,0,0,2,0,12,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,45,159,227,454,908
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,54,77,154,307
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4,14,20,40,80
3,0,0,0,0,0,4,0,0,0,0,4,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,3,0,2,0,0,46,161,230,460,920
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,13,47,67,134,268
5,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,18,63,90,180,360
6,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,2,0,10,0,0,0,1,0,0,0,0,0,0,1,0,3,0,1,41,145,207,414,828
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,21,30,60,120
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,14,20,40,80
9,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,14,49,70,140,280


Join the total restaurants column to the grouped dataframe.

In [57]:
extracted_col = rest["Total_Restaurants"]
hy_grouped = hy_grouped.join(extracted_col)
hy_grouped.head()

,Neighborhoods,ATM,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo,Total_Restaurants
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,2,1,1,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,2,0,0,2,3,0,12,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,2,1,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,908
1,A. S. Rao Nagar,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,307
2,Abhyudaya Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80
3,Abids,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,3,0,0,1,0,4,1,0,2,0,0,0,0,0,0,0,0,1,3,1,0,0,0,0,0,0,1,4,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,1,0,0,0,5,0,10,0,1,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,1,0,1,0,1,1,2,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,920
4,Adikmet,0,0,0,0,0,

### Total number of restaurants in all neighbourhoods in Hyderabad

In [60]:
Total = hy_grouped['Total_Restaurants'].sum()
Total

74708

- So there are 74708 restaurants in hyderabad(which it very high)

- So now we want to select a good location where the no of restaurants are less so that our chances of setting up a restaurant at that location  should be good

## Create a dataframe for shopping mall data only

In [61]:
hy_res = hy_grouped[["Neighborhoods","Total_Restaurants"]]
hy_res.head()

,Neighborhoods,Total_Restaurants
0,A. C. Guards,908
1,A. S. Rao Nagar,307
2,Abhyudaya Nagar,80
3,Abids,920
4,Adikmet,268


## Now cluster the neighbourhoods

Run k-means to cluster the neighborhoods in Hyderabad into 3 clusters

In [66]:
# set number of clusters
kclusters = 4

hy_clustering = hy_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hy_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 3, 0, 2, 3, 3, 2, 0, 0, 3], dtype=int32)

In [67]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hy_merged = hy_res.copy()

# add clustering labels
hy_merged["Cluster Labels"] = kmeans.labels_

In [68]:
hy_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hy_merged.head(10)

,Neighborhood,Total_Restaurants,Cluster Labels
0,A. C. Guards,908,2
1,A. S. Rao Nagar,307,3
2,Abhyudaya Nagar,80,0
3,Abids,920,2
4,Adikmet,268,3
5,Afzal Gunj,360,3
6,Aghapura,828,2
7,"Aliabad, Hyderabad",120,0
8,Alijah Kotla,80,0
9,Allwyn Colony,280,3


In [69]:
#Add latitude and longitude values by using the join operation(the new dataframe with the old dataframe containing the latitude and longitude values)
#hy_merged = hy_merged.join(hy_df.set_index("Neighborhood"), on="Neighborhood")

#print(hy_merged.shape)
hy_merged['Latitude'] = df_coords['Latitude']
hy_merged['Longitude'] = df_coords['Longitude']
hy_merged.head() # check the last columns!

,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
0,A. C. Guards,908,2,17.395015,78.459812
1,A. S. Rao Nagar,307,3,17.411200,78.508240
2,Abhyudaya Nagar,80,0,17.337650,78.564140
3,Abids,920,2,17.389800,78.476580
4,Adikmet,268,3,17.235790,78.541300


In [71]:
# sorting the results by Cluster Labels
print(hy_merged.shape)
hy_merged.sort_values(["Cluster Labels"], inplace=True)
hy_merged

(197, 5)


,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
196,Shivam Road,80,0,17.357390,78.473200
123,Mallapur,0,0,17.374930,78.515670
86,Kakatiya Nagar,80,0,17.428650,78.397620
153,Nawab Saheb Kunta,80,0,17.413990,78.345800
83,Jeedimetla,0,0,17.354420,78.432550
82,Jamia Osmania,188,0,17.342942,78.452441
81,Jalal Baba Nagar,0,0,17.504036,78.420122
80,Jahanuma,0,0,17.465740,78.372690
79,Jagadgirigutta,0,0,17.554030,78.344590
77,IDA Bollaram,0,0,17.448380,78.374250


In [72]:
hy_merged['Total_Restaurants'].max()

1862

## Now we visualize the resulting clusters

In [77]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hy_merged['Latitude'], hy_merged['Longitude'], hy_merged['Neighborhood'], hy_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [75]:
hy_merged.loc[hy_merged['Cluster Labels'] == 0]

,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
196,Shivam Road,80,0,17.357390,78.473200
123,Mallapur,0,0,17.374930,78.515670
86,Kakatiya Nagar,80,0,17.428650,78.397620
153,Nawab Saheb Kunta,80,0,17.413990,78.345800
83,Jeedimetla,0,0,17.354420,78.432550
82,Jamia Osmania,188,0,17.342942,78.452441
81,Jalal Baba Nagar,0,0,17.504036,78.420122
80,Jahanuma,0,0,17.465740,78.372690
79,Jagadgirigutta,0,0,17.554030,78.344590
77,IDA Bollaram,0,0,17.448380,78.374250


In [80]:
hy_merged.loc[hy_merged['Cluster Labels'] == 0].sum()

Neighborhood         Shivam RoadMallapurKakatiya NagarNawab Saheb K...
Total_Restaurants                                                 8045
Cluster Labels                                                       0
Latitude                                                   1741.473702
Longitude                                                  7847.864063
dtype: object

#### There are 8045 restaurants in the cluster 0 

### Cluster 1

In [81]:
hy_merged.loc[hy_merged['Cluster Labels'] == 1]

,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
61,Gachibowli,1503,1,17.474111,78.426392
166,Pisal Banda,1268,1,17.526770,78.252340
116,Madhapur,1862,1,17.525910,78.376330
48,Chintal Basti,1094,1,17.426750,78.512290
168,Punjagutta,1347,1,17.524900,78.449580
127,Masab Tank,1374,1,17.391860,78.459050
84,Jubilee Hills,1305,1,17.421967,78.525592
69,HITEC City,1678,1,17.363380,78.475020
13,Ameerpet,1415,1,17.533320,78.325290
174,Raj Bhavan Road,1347,1,17.505370,78.467490


In [82]:
hy_merged.loc[hy_merged['Cluster Labels'] == 1].sum()

Neighborhood         GachibowliPisal BandaMadhapurChintal BastiPunj...
Total_Restaurants                                                27155
Cluster Labels                                                      21
Latitude                                                    365.869451
Longitude                                                  1647.668476
dtype: object

#### There are 27155 restaurants in the cluster 1 

### Cluster 2

In [83]:
hy_merged.loc[hy_merged['Cluster Labels'] == 2]

,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
161,Pan bazar,815,2,17.462590,78.516300
151,Nanalnagar,878,2,17.423410,78.515340
152,Narayanguda,959,2,17.389310,78.467450
164,Patny,896,2,17.433790,78.492220
178,Rani Gunj,815,2,17.390320,78.545440
188,Sanghi Nagar,896,2,17.405390,78.471380
189,Santoshnagar,746,2,17.477175,78.528480
191,Secunderabad,776,2,17.396330,78.457048
149,"Nampally, Hyderabad",868,2,17.375379,78.498487
132,"Minister Road, Hyderabad",775,2,17.362310,78.471027


In [84]:
hy_merged.loc[hy_merged['Cluster Labels'] == 2].sum()

Neighborhood         Pan bazarNanalnagarNarayangudaPatnyRani GunjSa...
Total_Restaurants                                                23392
Cluster Labels                                                      56
Latitude                                                    487.747648
Longitude                                                  2197.378517
dtype: object

#### There are 23392 restaurants in the cluster 2

### Cluster 3

In [85]:
hy_merged.loc[hy_merged['Cluster Labels'] == 3]

,Neighborhood,Total_Restaurants,Cluster Labels,Latitude,Longitude
106,Lab quarters,388,3,17.512650,78.441290
32,Begum Bazaar,240,3,17.373580,78.469210
111,Lingojiguda,280,3,17.440300,78.527910
177,"Ramnagar, Hyderabad",307,3,17.511610,78.294110
29,Barkatpura,559,3,17.393870,78.498420
118,Maharajgunj,400,3,17.495380,78.333360
22,Bagh Lingampally,519,3,17.399310,78.499640
21,Badichowdi,560,3,17.388960,78.486810
19,Azampura,228,3,17.407100,78.502330
18,"Azamabad, Hyderabad",547,3,17.369170,78.436830


In [86]:
hy_merged.loc[hy_merged['Cluster Labels'] == 3].sum()

Neighborhood         Lab quartersBegum BazaarLingojigudaRamnagar, H...
Total_Restaurants                                                16116
Cluster Labels                                                     144
Latitude                                                    835.859953
Longitude                                                  3766.731447
dtype: object

#### There are 16116 restaurants in the cluster 3

# Final observation

Major number of restaurants are concentrated in the central area of Hyderabad city, with the highest number in cluster 1 & 2  and moderate number in cluster 3. This represents a great opportunity and high potential areas to open new restaurants as there is very little to no competition from existing restaurants. Meanwhile, restaurants in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighbourhoods in cluster 0 with relatively less competition. Property developers with unique selling propositions to stand out from the competition can also open new restaurants in neighbourhoods in cluster 3 with moderate competition. Lastly, property developers are advised to prefer central area of hyderabad city with the recommended cluster because lot of demand can be seen and with best business model can yeild a best restaurant.